In [1]:
import streamlit as st
from streamlit_folium import st_folium
import folium
from folium.plugins import Draw
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape
import geopandas as gpd
import numpy as np
import os

2024-09-17 09:46:19.889 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
# Set the page configuration
st.set_page_config(layout="wide")

st.title("Interactive World Population Map")

# Load the population raster data
raster_file = 'gpw_v4_population_count_rev11_2020_2pt5_min.tif'

if not os.path.exists(raster_file):
    st.warning(f"Population raster file '{raster_file}' not found. Please download and place it in the same directory.")
else:
    raster = rasterio.open(raster_file)

    # Get raster CRS
    raster_crs = raster.crs

    # Create a base map
    m = folium.Map(location=[20, 0], zoom_start=2)

    # Add the Draw plugin
    draw = Draw(export=True, filename='drawn_shape.geojson', position='topleft', draw_options={
        'polyline': False,
        'rectangle': False,
        'circle': False,
        'marker': False,
        'circlemarker': False,
        'polygon': True
    })
    draw.add_to(m)

    # Display the map
    output = st_folium(m, width=1000, height=600)

    # Process the drawn polygon
    if output['all_drawings']:
        # Get the last drawn feature
        last_feature = output['all_drawings'][-1]
        geom = last_feature['geometry']

        # Create a GeoDataFrame in WGS84
        polygon_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[shape(geom)])

        # Reproject the polygon to raster CRS
        polygon_proj = polygon_gdf.to_crs(raster.crs)

        # Mask the raster with the transformed polygon
        try:
            out_image, out_transform = mask(raster, polygon_proj.geometry, crop=True)
            out_image = out_image[0]  # Get the first band

            # Replace no-data values with zeros
            out_image = np.nan_to_num(out_image, nan=0)

            # Sum the population values
            total_population = np.sum(out_image)

            st.write(f"### Total population in the selected area: {int(total_population):,}")
        except Exception as e:
            st.error(f"Error processing the polygon: {e}")

2024-09-17 09:46:20.055 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 09:46:20.055 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 09:46:20.109 
  command:

    streamlit run c:\Users\915630\OneDrive - TRANSDEV\Documents\Personligt\Befolkningskarta\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-17 09:46:20.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 09:46:20.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 09:46:20.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
import streamlit as st
from streamlit_folium import st_folium
import folium
from folium.plugins import Draw
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import base64
from rasterio.enums import Resampling
from rasterio.transform import Affine
from rasterio.transform import array_bounds

ModuleNotFoundError: No module named 'matplotlib'